In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import transformers
import os

In [2]:
# MODEL

_LANGUAGE_         = 'es'
_PRETRAINED_LM_    = 'pysentimiento/robertuito-base-cased'
_PREPROCESS_TEXT_  = True
_TWEET_BATCH_SIZE_ = 5
_ADAPTER_CONFIG_   = transformers.ParallelConfig(mh_adapter = True, reduction_factor = 32)
_MAX_SEQ_LEN_      = 128
_OUTPUT_DIR_       = 'adapter_checkPoints'
_LOGGING_STEPS_    = 2
_NUM_AUTHORS_      = [8]
_K_FOLD_CV_        = 5

# TRAIN

_NO_GPUS_          = 2
_BATCH_SIZE_       = int(64 / _NO_GPUS_)
_EPOCHS_           = {'gender': 20, 'variety': 20}
_LEARNING_RATE_    = 1e-5

## Base

In [3]:
# LABEL DICTONARIES -----------------------------------------------------------------------

gender_dict    = {'female': 0, 
                  'male':   1}

varietyEN_dict = {'australia'    : 0,
                  'canada'       : 1,
                  'great britain' : 2,
                  'ireland'      : 3,
                  'new zealand'   : 4,
                  'united states': 5}

varietyES_dict = {'argentina': 0,
                  'chile'    : 1,
                  'colombia' : 2,
                  'mexico'   : 3,
                  'peru'     : 4,
                  'spain'    : 5,
                  'venezuela': 6}    

In [4]:
# SET LANGUAGE DIRECTORY

if _LANGUAGE_ == 'en':
    variety_dict = varietyEN_dict

elif _LANGUAGE_ == 'es':
    variety_dict = varietyES_dict

In [5]:
# SET LANGUAGE TOKENIZER
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(_PRETRAINED_LM_)
vocab = tokenizer.get_vocab()

## Dataset

In [6]:
from DatasetPAN17 import BasePAN17, DatasetPAN17, DatasetCrossVal

In [7]:
baseTrain  = BasePAN17(Dir             = '../data',
                      split            = 'train',
                      language         = _LANGUAGE_,
                      tokenizer        = tokenizer,
                      gender_dict      = gender_dict,
                      variety_dict     = variety_dict,
                      tweet_batch_size = _TWEET_BATCH_SIZE_,
                      max_seq_len      = _MAX_SEQ_LEN_,
                      preprocess_text  = _PREPROCESS_TEXT_)

baseTest  = BasePAN17(Dir             = '../data',
                      split            = 'test',
                      language         = _LANGUAGE_,
                      tokenizer        = tokenizer,
                      gender_dict      = gender_dict,
                      variety_dict     = variety_dict,
                      tweet_batch_size = _TWEET_BATCH_SIZE_,
                      max_seq_len      = _MAX_SEQ_LEN_,
                      preprocess_text  = _PREPROCESS_TEXT_)


Reading data...
    Done
Preprocessing text...
    Done
Tokenizing...
    Done
Merging data...
    Done

Total Instances: 84000


Reading data...
    Done
Preprocessing text...
    Done
Tokenizing...
    Done
Merging data...
    Done

Total Instances: 56000



In [8]:
Test = DatasetPAN17(Base_Dataset = baseTest, label = 'gender')

## Training

In [9]:
tasks = ['gender']

In [10]:
from transformers import AutoModelForSequenceClassification, AutoAdapterModel
from transformers import TrainingArguments, Trainer, AdapterTrainer, EarlyStoppingCallback
from TestingPAN17 import compute_accuracy, compute_test_metrics

num_v           = len(baseTest.variety_dict)
num_labels_dict = {"gender": 2, "variety": num_v,}

FewShot_Results = {}

for num in _NUM_AUTHORS_:
    # SHOW CURRENT PORTION
    print("Working with " + str(num) + " authors per label ... ")
    
    dataset_dict = {}
    models = {}
    
    for task_name in tasks:
        
        acc = []
        f1s = []
        
        for val_idx in range(_K_FOLD_CV_):
            print("Train,Val split number " + str(val_idx + 1) + " of " + str(_K_FOLD_CV_))
        
            # INITIALIZE MODEL-----------------------------------------
            models[task_name] = AutoModelForSequenceClassification.from_pretrained(
                _PRETRAINED_LM_,
                num_labels = num_labels_dict[task_name]
            )
            
            # GENERATES DATASET WITH CURRENT PORTION ----------------------
            
            data_train, data_val = baseTrain.cross_val(k = _K_FOLD_CV_, val_idx = val_idx, num_authors = num)
            Train = DatasetCrossVal(Base_Data = data_train, label = 'gender')
            Val   = DatasetCrossVal(Base_Data = data_val  , label = 'gender')

            # TRAIN ADAPTER--------------------------------------------

            training_args = TrainingArguments(
                learning_rate               = _LEARNING_RATE_,
                num_train_epochs            = _EPOCHS_[task_name],
                per_device_train_batch_size = _BATCH_SIZE_,
                per_device_eval_batch_size  = _BATCH_SIZE_,
                output_dir                  = _OUTPUT_DIR_ + '/' + task_name,
                save_total_limit            = 5,
                overwrite_output_dir        = True,
                remove_unused_columns       = False,
                evaluation_strategy         = 'epoch',
                logging_strategy            = 'epoch',
                save_strategy               = 'epoch',
                metric_for_best_model       = 'eval_acc',
                load_best_model_at_end      = True,
            )

            trainer = Trainer(
                model           = models[task_name],
                args            = training_args,
                train_dataset   = Train,
                eval_dataset    = Val,
                compute_metrics = compute_accuracy,
                callbacks       = [EarlyStoppingCallback(early_stopping_patience = 2, early_stopping_threshold = 0.0001)]
            )
            trainer.args._n_gpu = _NO_GPUS_

            trainer.train()

            # TEST MODEL ------------------------------------

            results = trainer.predict(Test)
            metrics = compute_test_metrics(baseTest, results.predictions, 'gender')
            
            acc.append(metrics['accuracy'])
            f1s.append(metrics['f1-score'])
           
        acc = np.array(acc)
        f1s = np.array(f1s)
        
        FewShot_Results[num] = {'accuracy': [acc.mean(), acc.std()], 'f1-score': [f1s.mean(), f1s.std()]}
        print("Results with " + str(num) + " authors per label: ", FewShot_Results[num])

Working with 8 authors per label ... 
Train,Val split number 1 of 5


Some weights of the model checkpoint at pysentimiento/robertuito-base-cased were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at pysentimiento/robertuito-base-cased and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.dense.bi

Epoch,Training Loss,Validation Loss,Acc
1,0.676100,0.699052,0.507619
2,0.622300,0.697157,0.526071
3,0.583300,0.696875,0.536012
4,0.529000,0.700067,0.541012
5,0.482100,0.707808,0.540893
6,0.428600,0.718779,0.543214
7,0.378600,0.731943,0.544107
8,0.348300,0.747850,0.544167
9,0.309000,0.765231,0.541845


***** Running Evaluation *****
  Num examples = 16800
  Batch size = 64
Saving model checkpoint to adapter_checkPoints/gender/checkpoint-5
Configuration saved in adapter_checkPoints/gender/checkpoint-5/config.json
Model weights saved in adapter_checkPoints/gender/checkpoint-5/pytorch_model.bin
Deleting older checkpoint [adapter_checkPoints/gender/checkpoint-10] due to args.save_total_limit
/001/usuarios/isaac.bribiesca/anaconda3/envs/NLP/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
***** Running Evaluation *****
  Num examples = 16800
  Batch size = 64
Saving model checkpoint to adapter_checkPoints/gender/checkpoint-10
Configuration saved in adapter_checkPoints/gender/checkpoint-10/config.json
Model weights saved in adapter_checkPoints/gender/checkpoint-10/pytorch_model.b

acc: 0.57: 100%|███████████████████████████████████████████████████████████████████| 2800/2800 [00:19<00:00, 142.62it/s]


...

loading configuration file https://huggingface.co/pysentimiento/robertuito-base-cased/resolve/main/config.json from cache at /001/usuarios/isaac.bribiesca/.cache/huggingface/transformers/3f85c0ee804baf604258892a88dd52cdf051d2418a511dcab7cab99a85a3a1b3.4cce50d5a926bf18fe43f2ea8d4596b505e97a64e6e700e993def66b06f1c83b
Model config RobertaConfig {
  "_name_or_path": "pysentimiento/robertuito-base-cased",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 130,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.21

Epoch,Training Loss,Validation Loss,Acc
1,0.598900,0.560749,0.706250
2,0.496900,0.578273,0.706964
3,0.405400,0.598350,0.713333
4,0.325800,0.654239,0.702440
5,0.253700,0.723390,0.710238


***** Running Evaluation *****
  Num examples = 16800
  Batch size = 64
Saving model checkpoint to adapter_checkPoints/gender/checkpoint-320
Configuration saved in adapter_checkPoints/gender/checkpoint-320/config.json
Model weights saved in adapter_checkPoints/gender/checkpoint-320/pytorch_model.bin
/001/usuarios/isaac.bribiesca/anaconda3/envs/NLP/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
***** Running Evaluation *****
  Num examples = 16800
  Batch size = 64
Saving model checkpoint to adapter_checkPoints/gender/checkpoint-640
Configuration saved in adapter_checkPoints/gender/checkpoint-640/config.json
Model weights saved in adapter_checkPoints/gender/checkpoint-640/pytorch_model.bin
/001/usuarios/isaac.bribiesca/anaconda3/envs/NLP/lib/python3.10/site-packages/torch/nn

acc: 0.8028571428571428: 100%|█████████████████████████████████████████████████████| 2800/2800 [00:17<00:00, 163.03it/s]

Results with 512 authors per label:  {'accuracy': [0.8098571428571427, 0.007987873462253879], 'f1-score': [0.8000596537338568, 0.013255013701296225]}


In [11]:
FewShot_Results

{8: {'accuracy': [0.6463571428571429, 0.038807741706089145],
  'f1-score': [0.675614778161259, 0.05229066275650628]},
 16: {'accuracy': [0.6705, 0.01659634825828275],
  'f1-score': [0.6892745171156573, 0.02207377870412438]},
 32: {'accuracy': [0.679142857142857, 0.008611809641772302],
  'f1-score': [0.6896027128017307, 0.006236413401855358]},
 48: {'accuracy': [0.6794285714285715, 0.013722095990312377],
  'f1-score': [0.6935243485276106, 0.025162403949095986]},
 64: {'accuracy': [0.6882857142857143, 0.03321113656345916],
  'f1-score': [0.6787820866714995, 0.048350822584644085]},
 128: {'accuracy': [0.7292142857142858, 0.007379646581881108],
  'f1-score': [0.7342845402829002, 0.013718943601915282]},
 256: {'accuracy': [0.7807857142857142, 0.012316821084705786],
  'f1-score': [0.775970202717284, 0.014919221734025953]},
 512: {'accuracy': [0.8098571428571427, 0.007987873462253879],
  'f1-score': [0.8000596537338568, 0.013255013701296225]}}